<img src="img/contour.png">
<img src="img/rgb_hsv.png">

In [1]:
import cv2
import numpy as np
from collections import deque #liste benzer circular queue

In [2]:
# nesne merkezini depolayacak veri tipi
buffer_size = 16 #  dequeuenun boyutu her zaman verilmeli.
pts = deque(maxlen = buffer_size) # 16 boyutlu bir list

In [3]:
# mavi renk aralığı HSV(Hue, Saturation, Value) = (Ton, Doygunluk, Parlaklık)
blueLower = (84,  98,  0)
blueUpper = (179, 255, 255)

In [4]:
# capture
cap = cv2.VideoCapture(0) # 480x960
cap.set(3, 960)
cap.set(4, 480)

True

In [6]:
while True:
    
    success, imgOriginal = cap.read() # okuyama başarılıysa success = True
    
    if success: 
        
        # blur
        blurred = cv2.GaussianBlur(imgOriginal, (11,11), 0) 
        
        # hsv
        hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)
        cv2.imshow("HSV Image",hsv)
        
        # mavi için maske oluştur
        mask = cv2.inRange(hsv, blueLower, blueUpper)
        cv2.imshow("mask Image",mask)
        # maskenin etrafında kalan gürültüleri sil
        mask = cv2.erode(mask, None, iterations = 2)
        mask = cv2.dilate(mask, None, iterations = 2)
        cv2.imshow("Mask + erozyon ve genisleme",mask)
        
        # farklı sürüm için
        # (_, contours,_) = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # kontur
        (contours,_) = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        center = None

        if len(contours) > 0: # mavi nesneler varsa
            
            # en buyuk konturu al
            c = max(contours, key = cv2.contourArea) # tüm mavi alanı alır.
            
            # dikdörtgene çevir 
            rect = cv2.minAreaRect(c) # konturu kapsayan minimum rectangle
            
            ((x,y), (width,height), rotation) = rect
            
            s = "x: {}, y: {}, width: {}, height: {}, rotation: {}".format(np.round(x), np.round(y), np.round(width), np.round(height), np.round(rotation))
            
            # kutucuk
            box = cv2.boxPoints(rect)
            box = np.int64(box)
            
            # moment # görüntünün merkezini bulmamızı sağlıyor
            M = cv2.moments(c)
            center = (int(M["m10"]/M["m00"]), int(M["m01"]/M["m00"])) # m10=1,0 point
            
            # konturu çizdir: sarı
            cv2.drawContours(imgOriginal, [box], 0, (0,255,255),2)
            
            # merkeze bir tane nokta çizelim: pembe
            cv2.circle(imgOriginal, center, 5, (255,0,255),-1)
            
            # bilgileri ekrana yazdır
            cv2.putText(imgOriginal, s, (25,50), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (255,255,255), 2)
            
            
        # deque
        pts.appendleft(center)
        
        for i in range(1, len(pts)):
            
            if pts[i-1] is None or pts[i] is None: continue
        
            cv2.line(imgOriginal, pts[i-1], pts[i],(0,255,0),3) # 
            
        cv2.imshow("Orijinal Tespit",imgOriginal)
        
    if cv2.waitKey(1) == ord("q"): 
        cv2.destroyAllWindows()
        break